# Adding Points with `+` in Python

This tiny notebook shows how **operator overloading** works by giving a custom class the ability to use the `+` operator.

**What you'll learn:**
- How a simple `Point(x, y)` class is defined
- How `__add__` lets you use `a + b`
- Why returning `NotImplemented` is helpful
- How `__repr__` controls the nice text you see when printing objects


In [1]:
# Define a simple 2D point and let it support +
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # a + b  ==>  a.__add__(b)
    def __add__(self, other):
        # If 'other' isn't a Point, let Python try the reverse add (other.__radd__(a))
        if not isinstance(other, Point):
            return NotImplemented
        return Point(self.x + other.x, self.y + other.y)

    def __repr__(self):
        return f"Point({self.x}, {self.y})"


## Demo: adding and chaining
Run the cell below. You'll see that `p1 + p2` returns a **new** `Point`, and chaining works because each `+` produces another `Point`.

In [2]:
# Demo
p1 = Point(2, 3)
p2 = Point(4, 1)
p3 = p1 + p2           # calls p1.__add__(p2)
print(p3)              # expected: Point(6, 4)

# Chaining works because __add__ returns a new Point
print(p1 + p2 + Point(10, -2))  # expected: Point(16, 2)


Point(6, 4)
Point(16, 2)


## Try it yourself
Edit the numbers and re-run to experiment. You can even create more `Point` objects and add them in different orders.

In [3]:
# Playground — change these values
a = Point(1, 1)
b = Point(5, -3)
c = Point(0, 10)
print("a + b =", a + b)
print("b + c =", b + c)
print("a + b + c =", a + b + c)


a + b = Point(6, -2)
b + c = Point(5, 7)
a + b + c = Point(6, 8)


## `__radd__` and mixed-type addition (e.g. `Point + 10`, `10 + Point`)

Sometimes you want to add a **number** to a `Point`. To make both `Point + 10` and `10 + Point` work, we:

1. Extend `__add__` to handle numbers (treat as adding to both `x` and `y`).
2. Implement `__radd__` for the reverse case when the **left** operand doesn't know how to add a `Point`.

If an operation isn't supported, we still return `NotImplemented` so Python can try other fallbacks (or raise a clear error).

In [4]:
# Upgraded Point with mixed-type addition and __radd__
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        # Point + Point
        if isinstance(other, Point):
            return Point(self.x + other.x, self.y + other.y)
        # Point + number (int/float) => add to both coordinates
        if isinstance(other, (int, float)):
            return Point(self.x + other, self.y + other)
        # Anything else: let Python try the reverse op or fail clearly
        return NotImplemented

    def __radd__(self, other):
        # number + Point should behave like Point + number
        if isinstance(other, (int, float)):
            return Point(self.x + other, self.y + other)
        return NotImplemented

    def __repr__(self):
        return f"Point({self.x}, {self.y})"

### Demo: numbers on either side, and `sum()`
- `Point(2,3) + 10` and `10 + Point(2,3)` both work now.
- `sum(points)` will also work because Python starts with `0 + first_point` which triggers `Point.__radd__`.

In [5]:
p = Point(2, 3)
print(p + 10)     # Point(12, 13)
print(10 + p)     # Point(12, 13)

points = [Point(1, 2), Point(3, 4), Point(5, -1)]
print(sum(points))           # Works via __radd__, but see note below
print(sum(points, Point(0,0)))  # Explicit start value is clearer and avoids relying on 0


Point(12, 13)
Point(12, 13)
Point(9, 5)
Point(9, 5)


> **Tip:** Although `sum(points)` works with our `__radd__`, using `sum(points, Point(0, 0))` is more explicit
and avoids confusing readers who might not know about the implicit `0 + first_point` step.